In [1]:
# Import libraries
import numpy as np
import pandas as pd
import tweepy
import psycopg2
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

In [2]:
# Keys
consumer_key = 'Eb7Q832xETPncAvbICrimtbgQ'
consumer_secret = 'myJ84Ry5Zete9zgqJtQb7pZK8fw9oAqF7CssGQAj0PXVdH5pP0'
access_token = '2760228716-kr6JayzHlTNbEf0JGmhaGdJzsVhXWqRU83nolpK'
access_secret = 'h7rgFGE0IrfQQXxotXuvhfAhZJq08aD7em6V5fo1fEO22'

In [3]:
# Twitter api
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [4]:
# Connect to database
conn_new = psycopg2.connect(dbname='conn_new2', user='postgres')
cursor = conn_new.cursor()

In [5]:
# Create table
cursor.execute("CREATE TABLE IF NOT EXISTS tweets (id SERIAL PRIMARY KEY, user_name VARCHAR, media BOOL, retweetcount INT, favcount INT, created_at TIMESTAMP, tweet text NOT NULL, location VARCHAR);")
conn_new.commit()

In [6]:
# Insert into table
try:
    for tweet in tweepy.Cursor(api.search,q="#nba -filter:retweets",tweet_mode='extended',count=100).items(17790):
        if 'media' in tweet.entities:
            media_bool = True
        else:
            media_bool = False
        cursor.execute("INSERT INTO tweets (user_name, media, retweetcount, favcount, created_at, tweet, location) VALUES (%s, %s, %s, %s, %s, %s, %s);", (tweet.author.screen_name, media_bool, tweet.retweet_count, tweet.favorite_count, tweet.created_at, tweet.full_text, tweet.user.location))
        conn_new.commit()        
except Exception as e:
         print(e)
        
cursor.close()

In [7]:
# Read from database
from sqlalchemy import create_engine
engine = create_engine('postgresql://postgres@localhost:5432/conn_new2')
df = pd.read_sql_query('select * FROM tweets',conn_new)

In [8]:
# Close connection to database
conn_new.close()

In [9]:
df

,id,user_name,media,retweetcount,favcount,created_at,tweet,location
0,1,kingyo_shiba,False,0,0,2019-02-19 13:19:05,渡邊雄太の今季の #NBA での最高得点は「 11得点～15得点 」と予想\n#スーパーチ...,
1,2,kingyo_shiba,False,0,0,2019-02-19 13:18:32,渡邊雄太は 今季 #NBA に「 16試合～20試合 」出場すると予想\n#スーパーチョイ...,
2,3,planetacb,False,0,0,2019-02-19 13:18:01,#NBA #Raptors\nLos Raptors casi fichan a Mirot...,"Comunidad de Madrid, España"
3,4,MomentousRealty,False,0,0,2019-02-19 13:17:37,https://t.co/a1cvBABuUK\n\n#LuxuryRealEstate #...,Real Estate Broker
4,5,SeeNewsEGY,False,0,0,2019-02-19 13:17:04,#NBA eyes to launch a new professional #basket...,Egypt
5,6,kingyo_shiba,False,0,0,2019-02-19 13:16:43,渡邊雄太は 今季中に #NBA のグリズリーズと「 本契約を結ぶ 」と予想\n#スーパーチョ...,
6,7,detikcom,False,0,2,2019-02-19 13:16:30,Misteri seputar IQOO mulai tersingkap. Smartph...,"Jakarta, Indonesia"
7,8,OrgulloVerde92,False,0,0,2019-02-19 13:14:58,¿Qué valoración haces del All Star 2019? / Wha...,España
8,9,GlobalSportsCtr,True,0,0,2019-02-19 13:13:23,How Patrick Corbin won the winter With a stagn...,NORTH AMERICA/EUROPE
9,10,timeofsports,False,1,0,2019-02-19 13:13:12,A teenager who became the youngest player to w...,NEW DELHI


In [10]:
total_tweets = len(df)

In [11]:
total_tweets

17790

In [12]:
unique_users = df.user_name.nunique()

In [13]:
unique_users

8863

In [14]:
media_tweets_num = len(df[df.media == True])

In [15]:
media_tweets_num

5426

In [16]:
most_retweeted = df[df.retweetcount == max(df.retweetcount)]

In [17]:
most_retweeted

,id,user_name,media,retweetcount,favcount,created_at,tweet,location
2389,2411,NBAHistory,False,1705,5725,2019-02-18 18:54:29,EVERY 50-POINT DUNK in #NBA Dunk Contest Histo...,


In [18]:
most_fav = df[df.favcount == max(df.favcount)]

In [19]:
most_fav

,id,user_name,media,retweetcount,favcount,created_at,tweet,location
2389,2411,NBAHistory,False,1705,5725,2019-02-18 18:54:29,EVERY 50-POINT DUNK in #NBA Dunk Contest Histo...,


In [20]:
# Replace blank locations
df['location'] = df.location.replace('', 'No location provided')

In [21]:
df.groupby('location').size().sort_values(ascending=False)

location
No location provided                                                                                                             4652
United States                                                                                                                     352
Newark, NJ                                                                                                                        256
Las Vegas, NV                                                                                                                     199
대한민국                                                                                                                              196
Los Angeles, CA                                                                                                                   167
Atlanta, GA                                                                                                                       157
Dallas, TX                                           

In [22]:
df.groupby('user_name').size().sort_values(ascending=False).head()

user_name
Fanteractive      254
sethrow991        231
NBANewsVideos     144
SportsWatch1      123
CaraotaDigital    116
dtype: int64

In [23]:
df.groupby('user_name')['retweetcount'].sum().sort_values(ascending = False).head(5)

user_name
NBAJPN           4003
NBAHistory       1705
arreazaortega     573
NBA               555
SC_ESPN           434
Name: retweetcount, dtype: int64

In [24]:
df.groupby('user_name')['favcount'].sum().sort_values(ascending = False).head(5)

user_name
NBAJPN           25826
NBAHistory        5725
NBA               2722
SkySportsNBA      2022
arreazaortega     1810
Name: favcount, dtype: int64

In [25]:
plotly.tools.set_credentials_file(username='gunajit23', api_key='ULuUce6nAp56xGWMIH2W')

In [26]:
till_hour = pd.to_datetime(df.created_at.dt.strftime('%Y-%m-%d-%H'))

In [27]:
trace = go.Scatter(
    x = df.groupby(till_hour).size().index,
    y = df.groupby(till_hour).size().values
)

data = [trace]

py.iplot(data, filename='tweets_num')

In [28]:
trace = go.Scatter(
    x = df[df.media == True].groupby(till_hour).size().index,
    y = df[df.media == True].groupby(till_hour).size().values
)

data = [trace]

py.iplot(data, filename='media_tweets_num')